# Optimization Experiment for DNN Architecture Sampling 

$$
N_{est}=\sum_{i=0}^{L-1} H_{i+1}(H_i + 1) \text{ where } L \in \mathbb{Z}^{++}, H_i \in \mathbb{Z}^{++}\\
\\
\\
\text{The value of }H_0\text{ and }H_L\text{ are known. Find the n combinations of }H_i\text{ for which all }  {i=0} \text{ to } {i={L-1}} \\ \text{eg. }(H_0, H_1, H_2, ..., H_L) \text{ } \text{where }N_{est}\text{ is closest to a given integer }N_T.
$$

In [1]:
import pdb
import numpy as np
from random import uniform
from gekko import GEKKO

# Experiment

In [2]:
# Architecture Parameters
L = 5
N_T = 1500
H_0 = 5
H_L = 1

# Sampling Constraints
H_min = 1
H_max = 1000

In [3]:
def n_params(H):
    N_est = 0
    for i in range(len(H)-1):
        N_est += H[i+1] * (H[i] + 1)
    return(N_est)

In [4]:
model = GEKKO()

In [5]:
H = [H_0]
for i in range(1,L-1):
    H.append(model.Var(value=round(uniform(H_min,H_max)), integer=True, lb=H_min, ub=H_max))
H.append(H_L)

In [6]:
model.Minimize(abs(N_T - n_params(H)))
model.options.SOLVER = 1
model.solver_options = ['minlp_maximum_iterations 10000',
                        'minlp_branch_method 3',
                        'minlp_max_iter_with_int_sol 500']
model.solve(disp=False)

In [7]:
H = np.hstack(H).astype(int).tolist()
H

[5, 36, 21, 22, 1]

In [8]:
n_params(H)

1500

In [9]:
model.cleanup()

---

# Architecture Sampling Function

In [10]:
def fc_num_params(H):
    '''
    Compute the total number of parameters in a fully connected neural network
    
    Args:
        H: List of neurons at each layer

    Returns:
        N: Number of parameters in the fully connected network
        
    Raises:
        -
        
    Example:
        >>> fc_num_params([5, 85, 7, 43, 1])
        1500
        
    Author:
        Dr. Calvin Chan
        calvin.chan@bayer.com
    '''

    N = 0
    for i in range(len(H)-1):
        N += H[i+1] * (H[i] + 1)
    return(N)

In [11]:
def parameters_sampling(num_params,
                        num_layers,
                        in_dim,
                        out_dim=1, 
                        n_min=1, 
                        n_max=None, 
                        n_samples=1, 
                        include_inout=True,
                        single_sample=False,
                        max_trials=1000):
    '''
    Randomly sampling DNN architecture based on give number of total parameters.
    This is use for neuron architecture sampling based on the same number of parameters given.
    
    Args:
        num_params: Total number of parameters to be distributed
        num_layers: Total number of layers
        n_min:      Minimum number of neurons per layer
        n_max:      Maximum number of neurons per layer
        in_dim:     Number of neurons at the input layer
        out_dim:    Number of neurons at the output layer
        n_samples:  Number of architecture samples to return (maximum number of samples return if there are less than demanded)
        include_inout: Flag indicate whether to include input and output layer neurons with samples
        max_trials: Maximum number of randomized trial for solution sampling if not enough samples found

    Returns:
        sample: List of elements splitted into multiple dimensions
        
    Raises:
        -
        
    Example:
        >>> parameters_sampling(num_params=500, 
                                num_layers=5,
                                in_dim=5,
                                out_dim=1, 
                                n_min=1, 
                                n_max=None, 
                                n_samples=10, 
                                include_inout=True,
                                single_sample=False,
                                max_trials=1000)
        [[5, 5, 11, 31, 1],
         [5, 6, 7, 46, 1],
         [5, 18, 9, 20, 1],
         [5, 19, 3, 65, 1],
         [5, 29, 5, 25, 1],
         [5, 29, 9, 5, 1],
         [5, 33, 7, 7, 1],
         [5, 49, 3, 11, 1],
         [5, 54, 1, 40, 1],
         [5, 63, 1, 19, 1]]
        
    Author:
        Dr. Calvin Chan
        calvin.chan@bayer.com
    '''
    
    samples = np.empty([0,num_layers],dtype=int)
    
    for trial in range(max_trials):
        # initialize MINLP (Mixed-Integer Nonlinear Programming Model)
        model = GEKKO()

        # Setup variables to be optimized
        H = [in_dim]
        for i in range(1,num_layers-1):
            # Randomize initial variables for diverse samples
            H.append(model.Var(value=round(uniform(H_min,H_max)), integer=True, lb=H_min, ub=H_max))
        H.append(out_dim)

        # Solving for Optimal Values
        model.Minimize(abs(num_params - n_params(H)))
        model.options.SOLVER = 1
        model.solver_options = ['minlp_maximum_iterations 10000',
                                'minlp_branch_method 3',
                                'minlp_max_iter_with_int_sol 500']
        model.solve(disp=False)
        
        # Check to make sure new sample is unique and aggregate it for output
        H = np.hstack(H).astype(int)
        H = H if include_inout else H[1:-1]
        samples = np.unique(np.vstack((samples, H)), axis=0)

        # Break the loop if we have enough samples
        if samples.shape[0] == n_samples:
            break

    # Convert from Numpy Array back to List
    samples = samples.tolist()
            
    return(samples)

In [12]:
test = parameters_sampling(num_params=500, 
                           num_layers=5,
                           in_dim=5,
                           out_dim=1, 
                           n_min=1, 
                           n_max=None, 
                           n_samples=10, 
                           include_inout=True,
                           single_sample=False,
                           max_trials=1000)

In [13]:
test

[[5, 1, 5, 69, 1],
 [5, 6, 7, 46, 1],
 [5, 9, 13, 21, 1],
 [5, 19, 15, 5, 1],
 [5, 33, 7, 7, 1],
 [5, 43, 5, 3, 1],
 [5, 49, 3, 11, 1],
 [5, 54, 1, 40, 1],
 [5, 54, 3, 2, 1],
 [5, 69, 1, 5, 1]]

---

## Testing

Source: https://apmonitor.com/wiki/index.php/Main/IntegerBinaryVariables

In [14]:
def f(x1,x2):
    return(4*x1**2-4*x2*x1**2+x2**2+x1**2-x1+1)

In [15]:
from gekko import GEKKO
m = GEKKO() # create GEKKO model
# create integer variables
x1 = m.Var(integer=True,lb=-5,ub=10)
x2 = m.Var(integer=True,lb=-1,ub=2)
m.Minimize(f(x1,x2))
m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0]))
print('x2: ' + str(x2.value[0]))

apm 35.157.216.221_gk_model11 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            2
   Intermediates:            0
   Connections  :            0
   Equations    :            1
   Residuals    :            1
 
 Number of state variables:              2
 Number of total equations: -            0
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              2
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj:  9.50E-01 Gap:       NaN
--Integer Solution:   1.00E+00 Low

Source: https://apmonitor.com/wiki/index.php/Main/OptionApmSolver